In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("jatinmehra/NegaBot-Product-Criticism-Catcher")
tokenizer = AutoTokenizer.from_pretrained("jatinmehra/NegaBot-Product-Criticism-Catcher")

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

2025-07-24 07:27:34.415475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753342054.592090      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753342054.646499      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

In [2]:
import pandas as pd
import numpy as np

synthetic_data = pd.read_csv("/kaggle/input/synthetic-data/product_sentiment_dataset_2k.csv")
synthetic_data.rename({'text':'tweet'}, axis=1, inplace=True)
synthetic_data.head()

,tweet,label
0,Such a smart mouse — it outsmarts me by refusi...,1
1,"This printer is so efficient at doing nothing,...",1
2,The performance of this vacuum cleaner is top-...,0
3,This fan is the Houdini of gadgets — it disapp...,1
4,"Absolutely love how this smartwatch turns on, ...",1


In [3]:
real = pd.read_csv("/kaggle/input/tweet-real/real.csv").drop(columns=["Unnamed: 0"])
real.head()

,id,label,tweet
0,907,1,My iPhone! It sucks! Keeps screwing up! Shut o...
1,3557,0,Flower Art #flower #art #colour #photography #...
2,2044,0,got my #ps4 with a copy of killzone shadow fal...
3,1265,0,Coffee Is Love ! #iphoneography #instagram #ip...
4,5338,0,I'm sure my iPhone just deleted every text mes...


In [4]:
df = pd.concat([synthetic_data, real]).drop(columns=['id'])
df.head()

,tweet,label
0,Such a smart mouse — it outsmarts me by refusi...,1
1,"This printer is so efficient at doing nothing,...",1
2,The performance of this vacuum cleaner is top-...,0
3,This fan is the Houdini of gadgets — it disapp...,1
4,"Absolutely love how this smartwatch turns on, ...",1


In [5]:
from sklearn.model_selection import train_test_split


train, valid = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [6]:
from datasets import Dataset

COLS = ['tweet', 'label']

train_df_clean = train[COLS].copy()
val_df_clean = valid[COLS].copy()

# Ensure labels are proper integers
train_df_clean['label'] = train_df_clean['label'].astype(np.int64)
val_df_clean['label'] = val_df_clean['label'].astype(np.int64)

# Reset index to ensure clean DataFrame structure
train_df_clean = train_df_clean.reset_index(drop=True)
val_df_clean = val_df_clean.reset_index(drop=True)

# Create datasets with explicit copy to avoid NumPy 2.0 issues
train_ds = Dataset.from_pandas(train_df_clean, preserve_index=False)
val_ds = Dataset.from_pandas(val_df_clean, preserve_index=False)

In [7]:
def tokenize(batch):
    """Tokenizes a batch of text inputs."""
    return tokenizer(batch["tweet"], truncation=True, max_length=256)

# Apply tokenization
train_ds = train_ds.map(tokenize, batched=True, remove_columns=['tweet'])
val_ds = val_ds.map(tokenize, batched=True, remove_columns=['tweet'])

Map:   0%|          | 0/2867 [00:00<?, ? examples/s]

Map:   0%|          | 0/717 [00:00<?, ? examples/s]

In [8]:

# Add a new padding token

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize the model's token embeddings to match the new tokenizer

model.resize_token_embeddings(len(tokenizer))

# Set the pad token id in the model's config

model.config.pad_token_id = tokenizer.pad_token_id

In [9]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

DIR = "/kaggle/working/"
# --- Training Arguments ---
training_args = TrainingArguments(
    output_dir=f"/kaggle/working/Chkpts",
    do_train=True,
    do_eval=True,
    eval_strategy="steps", # Evaluate every 'eval_steps'
    save_strategy="steps", # Save model every 'save_steps'
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=24,
    learning_rate=1e-5,
    logging_dir="./logs",
    logging_steps=200,
    save_steps=200,
    eval_steps=200,
    weight_decay=0.05,
    save_total_limit=1, # Only save the best model
    metric_for_best_model="accuracy",
    greater_is_better=True,
    load_best_model_at_end=True, # Load the best model found during training
    report_to="none", # Do not report to external services like Weights & Biases
    warmup_ratio=0.1, # 10% of total steps will be used for linear warmup
    lr_scheduler_type="cosine", # Use cosine learning rate decay
    dataloader_pin_memory=False, # Disable pin memory to avoid potential issues
)

In [10]:
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.special import softmax

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = softmax(logits, axis=-1)[:, 1] 
    preds = logits.argmax(axis=-1)
    
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, probs)
    
    return {
        "accuracy": acc,
        "roc_auc": auc
    }



data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=data_collator, 
    )
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Roc Auc
200,0.183300,0.085891,0.972106,0.995584
400,0.064300,0.095588,0.969317,0.995445


TrainOutput(global_step=540, training_loss=0.10341257607495344, metrics={'train_runtime': 317.6473, 'train_samples_per_second': 27.077, 'train_steps_per_second': 1.7, 'total_flos': 1257584431864320.0, 'train_loss': 0.10341257607495344, 'epoch': 3.0})

In [11]:
test = pd.read_csv("/kaggle/input/real-tweet-data/test_oJQbWVk.csv")

ds_test = Dataset.from_pandas(test[['tweet']])
ds_test = ds_test.map(tokenize, batched=True)

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

In [12]:
predictions = trainer.predict(ds_test)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

In [13]:
predicted_labels = probs.argmax(axis=1)
submission = pd.DataFrame({
    "id": test["id"],
    "label": predicted_labels 
})

submission.to_csv("submission.csv", index=False)

In [14]:
trainer.save_model("/kaggle/working/Best_model")